##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Object Detection


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/hub/tutorials/object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/hub/tutorials/object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/hub/tutorials/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/s?q=google%2Ffaster_rcnn%2Fopenimages_v4%2Finception_resnet_v2%2F1%20OR%20google%2Ffaster_rcnn%2Fopenimages_v4%2Finception_resnet_v2%2F1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub models</a>
  </td>
</table>

This Colab demonstrates use of a TF-Hub module trained to perform object detection.

## Setup


In [ ]:
# @title นำเข้าไลบรารีที่จำเป็น
# ส่วนนี้เป็นการนำเข้าไลบรารี (libraries) และกำหนดฟังก์ชันที่จำเป็นต่อการใช้งาน

# สำหรับรันงาน inference บน TF-Hub module
import tensorflow as tf           # ไลบรารีหลักสำหรับ Deep Learning/Machine Learning
import tensorflow_hub as hub      # ไลบรารีสำหรับเรียกใช้งานโมเดลสำเร็จรูปจาก TF-Hub

# สำหรับดาวน์โหลดรูปภาพและประมวลผลรูปภาพ
import matplotlib.pyplot as plt   # ไลบรารีสำหรับสร้างกราฟและแสดงผลรูปภาพ
import tempfile                   # ไลบรารีสำหรับจัดการไฟล์ชั่วคราว
from six.moves.urllib.request import urlopen  # สำหรับดาวน์โหลดข้อมูลจาก URL
from six import BytesIO           # สำหรับจัดการข้อมูลให้อยู่ในรูป Bytes

# สำหรับวาดรูปลงไปบนรูปภาพ
import numpy as np                # ไลบรารีสำหรับการคำนวณแบบเมทริกซ์และอาเรย์
from PIL import Image             # ไลบรารีสำหรับจัดการและประมวลผลรูปภาพ
from PIL import ImageColor        # สำหรับแปลงชื่อสีเป็นค่า RGB
from PIL import ImageDraw         # สำหรับวาดเส้นและรูปทรงบนรูปภาพ
from PIL import ImageFont         # สำหรับวาดตัวอักษรบนรูปภาพ
from PIL import ImageOps          # สำหรับปรับแต่งรูปภาพ เช่น ครอบ (crop), ใส่ขอบ (border)

# สำหรับวัดระยะเวลาในการทำ inference
import time

# พิมพ์เวอร์ชันของ TensorFlow
print(tf.__version__)

# ตรวจสอบว่า GPU ที่ใช้ได้มีอะไรบ้าง
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())


## ตัวอย่างการใช้

### ฟังก์ชันช่วยในการดาวน์โหลดรูปภาพ.

Code นี้ประยุกต์จาก [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) การใช้งานเบื้องต้น

In [ ]:
def display_image(image):
  """
  ฟังก์ชันสำหรับแสดงรูปภาพโดยใช้ matplotlib

  Arguments:
    image : รูปภาพที่อยู่ในรูปแบบ numpy array หรือรูปภาพในรูปแบบที่สามารถนำมาแสดงได้ด้วย plt.imshow
  """
  fig = plt.figure(figsize=(20, 15))  # กำหนดขนาดของ figure
  plt.grid(False)                     # ปิดการแสดงเส้นตาราง
  plt.imshow(image)                   # แสดงรูปภาพ


def download_and_resize_image(url, new_width=256, new_height=256, display=False):
  """
  ฟังก์ชันสำหรับดาวน์โหลดรูปภาพจาก URL และทำการปรับขนาดรูปภาพ

  Arguments:
    url        : URL ของรูปภาพที่ต้องการดาวน์โหลด
    new_width  : ความกว้างใหม่ของรูปภาพ (หน่วยเป็นพิกเซล)
    new_height : ความสูงใหม่ของรูปภาพ (หน่วยเป็นพิกเซล)
    display    : ถ้าเป็น True จะเรียกใช้ฟังก์ชัน display_image เพื่อแสดงรูปภาพ

  Returns:
    filename   : ชื่อไฟล์ (path) ที่บันทึกรูปภาพ
  """
  _, filename = tempfile.mkstemp(suffix=".jpg")  # สร้างไฟล์ชั่วคราวพร้อมนามสกุล ".jpg"
  response = urlopen(url)                        # เปิด URL เพื่อดาวน์โหลดไฟล์รูปภาพ
  image_data = response.read()                   # อ่านข้อมูลรูปภาพ
  image_data = BytesIO(image_data)               # แปลงข้อมูลให้อยู่ในรูป BytesIO
  pil_image = Image.open(image_data)             # เปิดรูปภาพด้วย PIL
  # ปรับขนาดรูปภาพให้พอดีกับ (new_width, new_height) โดยใช้ ImageOps.fit และ Image.LANCZOS
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.LANCZOS)
  pil_image_rgb = pil_image.convert("RGB")       # แปลงรูปเป็นโหมด RGB
  pil_image_rgb.save(filename, format="JPEG", quality=90)  # บันทึกเป็นไฟล์ JPEG
  print("Image downloaded to %s." % filename)    # แสดงข้อความไฟล์ที่บันทึก
  if display:
    display_image(pil_image)                     # ถ้า display=True ให้เรียกฟังก์ชันแสดงรูปภาพ
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """
  ฟังก์ชันสำหรับวาดกรอบสี่เหลี่ยม (bounding box) พร้อมข้อความกำกับ ลงบนรูปภาพ

  Arguments:
    image           : รูปภาพที่ต้องการวาดกรอบ
    ymin, xmin      : ค่า y และ x ของจุดเริ่มต้นด้านบนซ้ายของกรอบ (normalized value ระหว่าง 0-1)
    ymax, xmax      : ค่า y และ x ของจุดสิ้นสุดด้านล่างขวาของกรอบ (normalized value ระหว่าง 0-1)
    color           : สีของกรอบ (ค่า string หรือค่า RGBA)
    font            : ฟอนต์ที่ใช้ในการเขียนข้อความ
    thickness       : ความหนาของเส้นกรอบ
    display_str_list: รายการข้อความที่ต้องการแสดงภายในกรอบ
  """
  draw = ImageDraw.Draw(image)       # สร้างวัตถุสำหรับวาด (Draw object) บนรูปภาพ
  im_width, im_height = image.size   # ได้ขนาดความกว้างและความสูงของรูปภาพ
  # คำนวณตำแหน่งขอบซ้าย, ขวา, บน, ล่าง ของกรอบ ให้อยู่ในหน่วยพิกเซล
  (left, right, top, bottom) = (xmin * im_width,
                                xmax * im_width,
                                ymin * im_height,
                                ymax * im_height)
  # วาดเส้นกรอบสี่เหลี่ยมด้วย draw.line
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top), (left, top)],
            width=thickness, fill=color)

  # คำนวณความสูงของข้อความแต่ละอันเพื่อจัดวางข้อความไม่ให้เลยขอบของกรอบ
  display_str_heights = [font.getbbox(ds)[3] for ds in display_str_list]
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  # เลือกตำแหน่งวางข้อความว่าควรอยู่เหนือหรือใต้ขอบบนของกรอบ
  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height

  # วนลูปข้อความจากอันหลังสุดไปยังอันแรก เพื่อนำมาเรียงต่อกัน
  for display_str in display_str_list[::-1]:
    bbox = font.getbbox(display_str)     # ได้ขนาดของ bounding box ของข้อความ
    text_width, text_height = bbox[2], bbox[3]
    margin = np.ceil(0.05 * text_height) # เว้นขอบด้านบนและล่าง
    # วาดพื้นหลังสีเดียวกับกรอบ เพื่อให้ข้อความอ่านง่าย
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    # วาดข้อความลงบนรูปภาพ
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    # ปรับตำแหน่ง text_bottom สำหรับข้อความถัดไป
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """
  ฟังก์ชันสำหรับวาดกรอบ (bounding box) พร้อมชื่อคลาสและคะแนนความเชื่อมั่นลงบนรูปภาพ

  Arguments:
    image       : รูปภาพ (numpy array) ที่ต้องการวาดกรอบ
    boxes       : อาเรย์ของพิกัดกรอบ bounding box (ymin, xmin, ymax, xmax) ในรูปแบบ normalized
    class_names : อาเรย์หรือ list ของชื่อคลาสที่ตรวจจับได้
    scores      : อาเรย์ของคะแนนความเชื่อมั่น (confidence score)
    max_boxes   : จำนวนกรอบสูงสุดที่จะวาดลงบนรูปภาพ
    min_score   : เกณฑ์คะแนนขั้นต่ำของความเชื่อมั่น (confidence score) ที่จะนำมาวาด
  """
  # สร้างลิสต์สีทั้งหมดที่มีใน ImageColor.colormap
  colors = list(ImageColor.colormap.values())

  # พยายามโหลดฟอนต์จาก path ที่กำหนด
  try:
    font = ImageFont.truetype(
      "/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
      25
    )
  except IOError:
    # ถ้าไม่มีฟอนต์ดังกล่าว ให้ใช้ฟอนต์เริ่มต้นของ PIL
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  # วนลูปกรอบทั้งหมด แต่ไม่เกิน max_boxes
  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:  # ถ้าคะแนนมากกว่าหรือเท่ากับเกณฑ์ขั้นต่ำ
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      # แปลงชื่อคลาส (bytes) เป็น string แล้วคำนวณเปอร์เซ็นต์คะแนน
      display_str = "{}: {}%".format(
          class_names[i].decode("ascii"),
          int(100 * scores[i])
      )
      # เลือกสีของกรอบ (hash ของชื่อคลาส mod จำนวนสีที่มี)
      color = colors[hash(class_names[i]) % len(colors)]
      # แปลง numpy array ให้เป็นรูปภาพแบบ PIL เพื่อวาดกรอบ
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str]
      )
      # หลังจากวาดกรอบเสร็จ ให้อัปเดตอาเรย์ภาพเดิมด้วยข้อมูลจาก image_pil
      np.copyto(image, np.array(image_pil))

  return image


## การใช้งานโมดูล

โหลดรูปภาพสาธารณจาก  Open Images v4 บันทึกลงในเครื่อง (Colab) , และแสดงผล

In [ ]:
# By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
# กำหนด URL ของรูปภาพที่ต้องการดาวน์โหลด
image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg"  #@param

# ดาวน์โหลดรูปภาพและปรับขนาดเป็น 1280 x 856 โดยการเรียกฟังก์ชัน download_and_resize_image
# และสั่งให้แสดงรูปภาพหลังดาวน์โหลด (display=True)
downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)


เลือก โมเดล Object detection และใช้กับรูปภาพที่ดาวน์โหลดมา
 โมดูลมีให้เลือกคือ:
* **FasterRCNN+InceptionResNet V2**: มีความแม่นยำสูง,
* **ssd+mobilenet V2**: ขนาดเล็กและทำงานเร็ว.

In [ ]:
# module_handle คือ URL ของโมเดลที่ต้องการโหลดจาก TF-Hub
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"  #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

# โหลดโมเดลจาก TF-Hub โดยใช้ module_handle และเข้าถึง signature ชื่อ 'default' เพื่อใช้งานโมเดล
detector = hub.load(module_handle).signatures['default']


In [ ]:
def load_img(path):
  """
  ฟังก์ชันสำหรับโหลดรูปภาพจากไฟล์

  Arguments:
    path : ที่อยู่ของไฟล์รูปภาพ (path หรือ URL ของรูปภาพใน local)

  Returns:
    img  : Tensor ที่แสดงถึงรูปภาพในรูปแบบ RGB
  """
  img = tf.io.read_file(path)             # อ่านไฟล์รูปภาพจาก path
  img = tf.image.decode_jpeg(img, channels=3)  # แปลงข้อมูลรูปภาพเป็น JPEG โดยกำหนดให้มี 3 ช่องสี (RGB)
  return img


In [ ]:
def run_detector(detector, path):
  """
  ฟังก์ชันสำหรับรันโมเดลตรวจจับวัตถุ (object detection) บนรูปภาพที่กำหนด
  และแสดงผลการตรวจจับพร้อมกรอบสี่เหลี่ยม (bounding boxes) และชื่อวัตถุบนรูปภาพ

  Arguments:
    detector : โมเดลที่โหลดมาจาก TF-Hub ซึ่งมี signature 'default' สำหรับตรวจจับวัตถุ
    path     : path หรือที่อยู่ของไฟล์รูปภาพที่จะทำการตรวจจับ
  """
  # โหลดรูปภาพจากไฟล์
  img = load_img(path)

  # แปลงค่าพิกเซลของรูปภาพให้อยู่ในช่วง [0,1] และเพิ่มมิติ batch ด้านหน้า [tf.newaxis, ...]
  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

  # เริ่มต้นจับเวลาสำหรับการทำ inference
  start_time = time.time()
  # ส่งรูปภาพ (converted_img) เข้าโมเดล detector เพื่อให้โมเดลตรวจจับวัตถุ
  result = detector(converted_img)
  # สิ้นสุดการจับเวลา
  end_time = time.time()

  # แปลงผลลัพธ์ที่ได้จาก Tensor ให้เป็น numpy array (เพื่อให้ง่ายต่อการประมวลผลต่อ)
  result = {key: value.numpy() for key, value in result.items()}

  # พิมพ์จำนวนวัตถุที่ตรวจจับได้ และเวลาที่ใช้ในการทำ inference
  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time - start_time)

  # วาดกรอบสี่เหลี่ยม (bounding boxes) และชื่อวัตถุบนรูปภาพ
  image_with_boxes = draw_boxes(
      img.numpy(),
      result["detection_boxes"],
      result["detection_class_entities"],
      result["detection_scores"]
  )

  # แสดงรูปภาพที่มีกรอบสี่เหลี่ยมและชื่อวัตถุ
  display_image(image_with_boxes)


In [ ]:
# เรียกใช้ฟังก์ชัน run_detector เพื่อรันโมเดลตรวจจับวัตถุบนรูปภาพที่ดาวน์โหลดมา
run_detector(detector, downloaded_image_path)


### ลองกับรูปอื่นๆ
ตัวอย่างการทดสอบโมเดลกับรูปภาพอื่น ๆ หลายใบ พร้อมดูเวลาที่ใช้ในการทำ inference


In [ ]:
# กำหนดรายการ URL ของรูปภาพเพิ่มเติม เพื่อนำมาทำการตรวจจับวัตถุ (object detection)
image_urls = [
    # Source: https://commons.wikimedia.org/wiki/File:The_Coleoptera_of_the_British_islands_(Plate_125)_(8592917784).jpg
    "https://upload.wikimedia.org/wikipedia/commons/1/1b/The_Coleoptera_of_the_British_islands_%28Plate_125%29_%288592917784%29.jpg",
    # By Américo Toledano, Source: https://commons.wikimedia.org/wiki/File:Biblioteca_Maim%C3%B3nides,_Campus_Universitario_de_Rabanales_007.jpg
    "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg",
    # Source: https://commons.wikimedia.org/wiki/File:The_smaller_British_birds_(8053836633).jpg
    "https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg",
]

def detect_img(image_url):
  """
  ฟังก์ชันสำหรับ:
    1) ดาวน์โหลดและปรับขนาดรูปภาพจาก image_url
    2) รันโมเดลตรวจจับวัตถุ (detector) บนรูปภาพดังกล่าว
    3) แสดงผลการตรวจจับและเวลาที่ใช้ในการทำ inference

  Arguments:
    image_url : URL ของรูปภาพที่ต้องการนำมาทำการตรวจจับ
  """
  start_time = time.time()  # เริ่มต้นจับเวลาการประมวลผล

  # ดาวน์โหลดรูปภาพและปรับขนาดเป็น 640 x 480 พิกเซล
  image_path = download_and_resize_image(image_url, 640, 480)

  # รันโมเดลตรวจจับวัตถุที่โหลดไว้ก่อนหน้า (detector)
  run_detector(detector, image_path)

  end_time = time.time()  # สิ้นสุดการจับเวลาการประมวลผล

  # แสดงผลเวลา (หน่วยเป็นวินาที) ที่ใช้ในการทำ inference
  print("Inference time:", end_time - start_time)

In [ ]:
detect_img(image_urls[0])

In [ ]:
detect_img(image_urls[1])

In [ ]:
detect_img(image_urls[2])

**ให้นักศึกษาทดลองนำรูปภาพของตัวเองมาทดลองทำ Object Detection จากตัวอย่าง Code ด้านบน**

In [ ]:
### เขียน Code สำหรับ Object detection รูปของตัวเอง
